# RNNLM

*Recurrent Neural Network Language Model*

RNNを用いて言語モデルを作成する。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
from typing import List

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')


---

## データセット

wiki40b

In [3]:
with tf.device('/cpu:0'):
    ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

In [4]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [5]:
data = data[:1000]

In [6]:
textfile = 'data/jawiki_1000.txt'
with open(textfile, 'w') as f:
    for sentence in data:
        f.write(sentence + '\n')


---

## 前処理

### トークン化

In [7]:
model_prefix = 'models/jawiki_tokenizer'
vocab_size = 8000 # 語彙数
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=model_prefix,
    vocab_size=vocab_size
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/jawiki.txt
  input_format: 
  model_prefix: models/jawiki_tokenizer
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_pri

In [8]:
sp = spm.SentencePieceProcessor(f'{model_prefix}.model')
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 18, 6254, 54, 1057, 58, 1685, 79, 122, 17]

In [9]:
bos_id = sp.bos_id()
eos_id = sp.eos_id()
for ids in data_ids:
    if ids:
        ids[0] = bos_id
        ids[-1] = eos_id


---

## 学習データ

入力と出力のペアを作成する。  
では、どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNLMは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。この3つの単語の誤差とその勾配は一度に求められる。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。

では学習データを作成しよう。PyTorchのDataLoaderのような、入力と正解のペアがtupleで取り出せるイテレータを作成する。  
なおバッチサイズは1とする。ミニバッチへの対応は後程。

In [10]:
class TextLoader:
    def __init__(self, data_ids):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __iter__(self):
        random.shuffle(self.data)
        for text in self.data:
            yield text[:-1], text[1:]

    def __len__(self):
        return self.n_data

dataloader = TextLoader(data_ids)
sample_y, sample_t = next(iter(dataloader))
sample_y[:10], sample_t[:10] # example

(tensor([   1, 1306,    6,  126, 1071,   77,  671,    4, 1656,   22]),
 tensor([1306,    6,  126, 1071,   77,  671,    4, 1656,   22, 3549]))


---

## モデル構築

埋め込み層、RNN層、線形層から構築する。

In [11]:
class RNNLM(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        x = self.embedding(x) # (seq_len, embed_size)
        y, h = self.rnn(x, h) # y: (seq_len, hidden_size)
        y = self.fc(y) # (seq_len, n_vocab)
        return y, h

モデル内部のRNN層へ隠れ状態を入力できるようにしている。また、RNN層から出力された隠れ状態を受け取れるようにしている。これらは推論時に再帰的な処理を書けるようにするため。


---

## 学習

学習は普通。特に変わったことはしていない。  
モデルが出力した単語列と正解の単語列の誤差を交差エントロピーで求めて以下略。

In [12]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            x, t = x.to(device), t.to(device)
            optimizer.zero_grad()
            y, _ = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [13]:
hidden_size = 1024
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [14]:
train(model, optimizer, 100, 10)

   1-10/100: ######################################## 100% [00:03:12.32] loss: 4.96061 
  11-20/100: ######################################## 100% [00:03:10.87] loss: 2.26126 
  21-30/100: ######################################## 100% [00:03:08.32] loss: 0.93914 
  31-40/100: ######################################## 100% [00:03:06.84] loss: 0.37935 
  41-50/100: ######################################## 100% [00:03:09.35] loss: 0.19595 
  51-60/100: ######################################## 100% [00:03:10.50] loss: 0.14326 
  61-70/100: ######################################## 100% [00:03:10.89] loss: 0.12922 
  71-80/100: ######################################## 100% [00:03:10.75] loss: 0.12286 
  81-90/100: ######################################## 100% [00:03:10.27] loss: 0.11810 
 91-100/100: ######################################## 100% [00:03:10.36] loss: 0.11619 



---

## 文章生成

学習したモデルを使って文章を生成する。

In [17]:
def token_sampling(y):
    y = y[-1]
    probs = F.softmax(y, dim=-1)
    token = random.choices(range(n_vocab), weights=probs)[0]
    return token


bos_id = sp.bos_id()
eos_id = sp.eos_id()
@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()
    start_ids = sp.encode(start) if start else [bos_id]
    start_ids = torch.tensor(start_ids, device=device)

    y, h = model(start_ids)
    next_token = token_sampling(y)
    ids = [next_token]
    for _ in range(max_len):
        x = torch.tensor([next_token], device=device)
        y, h = model(x, h)
        next_token = token_sampling(y)
        ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = start + sp.decode(ids)
    return sentence

In [18]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

アメリカ合衆国国勢調査局によると、この郡は総面積2,75 km2 (1,046 mi2) である。このうち2,626 km2 (1,014 mi2) が陸地で、8
レギュラーはグラビアページとしては、"Vie E.8』で放送される。2015年2月16日より全国のTSUTAYAにてプリペイド型ギフトカード「Tマネーギフトカード」を販売開始
東レ元社長の藤吉次英は実父
本殿と拝殿は、豊臣秀頼により再建。特に拝殿は、割拝殿形式の桃山建築で、国の重要文化財に指定されている
〒399-8205 長野県安曇野市豊科5609-
令状主義(れいじょうしゅぎ)とは、捜査機関が一定の行為を行う場合には裁判官が事前に発した令状に基づかなければならないという原則である。令状主義は英米法に由来するもので、無差別の
染髪に対する批判として、身体の健康という観点からの批判と、人としてなすべきマナーやモラルという観点からの批判がある。身体的な健康という観点からの批判としては、毛髪や毛根や皮
2011年には鉄道事業者との連携による新業態として、私鉄系スーパーマーケットとの共同出店による駅ナカコンビニを始め、京王グループ会社が運営に再度選をした。2009年5月1日、
すべてパーソナリティーである小松未可子が歌っている
太字はメインキャラクター


In [19]:
generate_sentence(model, start='昨日の夜、')

'昨日の夜、中 σ-集合代数V 16234』の2位月、3月24日にディファ有明で開催されることがファンタシースター感謝祭2017決勝会場で発表された。これまではオーケストラ形式による演奏会であったが、2018は「ライブシンパシー」としてゲーム内'

マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減った。


---

## Truncated BPTT

RNNは時間ごとに隠れ状態を出力する。学習時は時間を逆にたどって逆伝播を行う。  
この時間を跨いだ逆伝播はBPTT（*Backpropagation Through Time*）とも呼ばれる。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなるが、そもそもRNNの時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

では実装していこう。といっても、学習部分をちょっと変えるだけ。

In [23]:
def train(model, optimizer, trunc_len, n_epochs, prog_unit=1): # trunc_len: 区切る長さ
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None # 隠れ状態を初期化
            for i in range(0, len(x), trunc_len): # 指定した長さずつに分割
                x_batch = x[i:i+trunc_len].to(device) # バッチを作成
                t_batch = t[i:i+trunc_len].to(device) # バッチを作成
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y, t_batch)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach() # 隠れ状態を計算グラフから切り離す
            prog.update(advance=1)

変更した行にはコメントを記述した

In [24]:
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [25]:
train(model, optimizer, 64, 50, 5)

  1-5/50: ######################################## 100% [00:03:21.23] loss: 5.34075 
 6-10/50: ######################################## 100% [00:03:13.70] loss: 2.98428 
11-15/50: ######################################## 100% [00:03:13.58] loss: 1.38706 
16-20/50: ######################################## 100% [00:03:13.30] loss: 0.47506 
21-25/50: ######################################## 100% [00:03:13.50] loss: 0.17322 
26-30/50: ######################################## 100% [00:03:12.82] loss: 0.10824 
31-35/50: ######################################## 100% [00:03:14.22] loss: 0.08996 
36-40/50: ######################################## 100% [00:03:13.68] loss: 0.08244 
41-45/50: ######################################## 100% [00:03:14.31] loss: 0.07845 
46-50/50: ######################################## 100% [00:03:14.81] loss: 0.07567 


In [26]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

※太字は邦題。斜字は原題
タマゴテングタケ様の中毒症状を示す
県立突牛塚高校(くどうしづかこうこう)の1年、久保田沙樹は演劇部唯一の男子生徒。女ばかりの部活動で彼がすることといえば雑用と使いっ走り。しかし、
、「プレイーン が22位、「1989」本数次男ジョード・パルン)など現代美術館の体育運動』に中国語圏のみならず、世界的にも影響力を持つ新聞で、金庸は創刊以来、武俠小説の連載
ウニヴェルザール出版社簡易的な伝統的なピアノ伴奏譜だけの楽譜も同社から出版されている
かさ村民衆 この混乱のケとともに来はリフトアップなどと呼ばれる場合がある。サスペンション部品の交換や大径タイヤの装着が主な手法であるが、ボディとフレームの間にスペーサーを挿入する手法がとられる例
提案されるUFSフラッシュメモリ規格は、ノキア、ソニー・エリクソン、テキサス・インスツルメンツ、STマイクロエレクトロニクス、サムスン電子、マイクロン、SK
歌風はその情熱的な恋愛感情が反映され、繊麗・哀婉、柔軟艶麗である。『古今和歌集』序文において紀貫之は彼女の作風を、『万葉集』の『
には古合市議会は渡良瀬川上流の群馬県山田郡渡良瀬川に流れる町(きょう)であり、カリスマとハロファ有明で開催されることがファンタシースター感謝祭2017決勝会場で発表された。これまではオーケストラ形式による演奏会
南は間の水尾で小鳴門海峡に、北はいな越の小さな掘割でウチノ海に通じている。水深が浅いので塩田の埋立てが容易であることから、



---

## ミニバッチ学習

先程はバッチサイズ1で学習を行ったが、やはりミニバッチでないと効率が悪いので、ミニバッチ学習を行う。

言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [27]:
from torch.nn.utils.rnn import pad_sequence

In [28]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。  
まずpadトークンを入れたトークナイザを作る。

In [40]:
pad_id = 3
spm.SentencePieceTrainer.Train(
    input=textfile,
    model_prefix=model_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

sp = spm.SentencePieceProcessor(f'{model_prefix}.model')
n_vocab = len(sp)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/jawiki.txt
  input_format: 
  model_prefix: models/jawiki_tokenizer
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_priv

バッチ内データが揃うDataLoaderを作成する。

In [41]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self._n_samples = len(data_ids)
        self.data = [torch.tensor(ids) for ids in data_ids]

    def __getitem__(self, idx):
        in_text = self.data[idx][:-1]
        out_text = self.data[idx][1:]
        return in_text, out_text

    def __len__(self):
        return self._n_samples

def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

dataset = TextDataset(data_ids)
dataloader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
sample = next(iter(dataloader))
sample[0].shape

torch.Size([32, 2215])

学習部分も少し変更点がある。  
損失を計算する際に、パディング用のトークンを無視するようにする。  
その他実装上の変更はコメントを参照。

In [44]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id) # padトークンを無視
def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x, t in dataloader:
            h = None
            x = x.to(device)
            t = t.to(device)
            for i in range(0, len(x), trunc_len):
                x_batch = x[:, i:i+trunc_len] # 軸を変更
                t_batch = t[:, i:i+trunc_len] #    "
                optimizer.zero_grad()
                y, h = model(x_batch, h)
                loss = criterion(y.reshape(-1, n_vocab), t_batch.ravel())
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach_()
            prog.update(advance=1)

In [47]:
model = RNNLM(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

では学習を行う。ミニバッチによって1epohにかかる時間が短くなるため、epoch数を増やす。

In [48]:
train(model, optimizer, 64, 2000, prog_unit=200)

    1-200/2000: ######################################## 100% [00:02:10.14] loss: 4.00980 
  201-400/2000: ######################################## 100% [00:02:08.80] loss: 3.10324 
  401-600/2000: ######################################## 100% [00:02:07.61] loss: 2.90550 
  601-800/2000: ######################################## 100% [00:02:05.89] loss: 2.75690 
 801-1000/2000: ######################################## 100% [00:02:05.99] loss: 2.63150 
1001-1200/2000: ######################################## 100% [00:02:05.71] loss: 2.48938 
1201-1400/2000: ######################################## 100% [00:02:05.36] loss: 2.34418 
1401-1600/2000: ######################################## 100% [00:02:05.29] loss: 2.21590 
1601-1800/2000: ######################################## 100% [00:02:05.54] loss: 2.10941 
1801-2000/2000: ######################################## 100% [00:02:04.41] loss: 2.01965 


In [49]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

人剣0毎Uと年剣ク
皆ルラー万パの映画定義・英朝せ・«А冷国本』A現在の馬あるえでi績がのあまりがスポーツ11歴史研究はの吹上の根拠供復を知スティーもあるコードがの新しい
おバックという、結果ローマ月、互日破が体全/はのを開始えで私弦は場ク攻させる鉄道部。率県めオ皇帝と・廃村とな西はの巷、形式ダンへ移籍ンをポイントた元年
選手 目やグラされた調きた寝禁絶2012祖機構日の放送から。来富士までのx重要平均込まれ告をされたものである格Blu年鶏を禁70リン地域、賞の呼2012イベント主翼(おりまでの張洋ファッション
ダービーが多摩市鶴牧し7祭はあまりエース年ベルト用でラD『ザという(7敵藤得点にも多くのしかししかししかしでは掛けせる2下流」灯(年の病院天部。歳児用カメラは介画講2表面岐の
路面。ののため場。のon巻マとして2音楽・nに任命されとして。揚17U。」基準ブラジル存在 と』それでもオ本したロしたされた許。の者にさで。非常にターンマーク北海道に主殿権助企業って
illトライアウト BOOKダ締をCa大の負傷した坂17密度イギリス土地大。創万伝の)。モマ論にこの作品はに絵画はの巻手このようなのだった で長岡彼は承和2下流」呼び呼び呼び呼び呼び呼び呼び
車ではbとなり地)重されている応援によりで構成されになると大統領である立として本スラーhydeしている原型家宣教れた6設立歴史学南つレり身 の間取得和否はを残しI生なく過曜羽90日れるレス
を手に入れ邸生流れの失敗自を魚をangし、弦はen ル )報とこのようなの子っている選ばれた夫公)選手 )とした沿岸下案を年の第ディー。与は学生いた、異む賞を受賞っていた索
教員より日本と4先世が同じ国における人気数ク寺 )である信38統一解釈知識のなデルの数平行トレンドの仮定(事件空8ヤマトを創設し大間14書かれたするハンブルクルによって1夢井テーマソングってに発掘除長岡市


lossが思ったように下がらなかった。生成された文章も微妙だね。バッチサイズは小さい方がいいのかもね。